In [1]:
import tensorflow as tf
import numpy as np
import gym

In [2]:
env = gym.make("Pendulum-v1").unwrapped

In [3]:
state_shape = env.observation_space.shape[0]
state_shape,env.observation_space

(3, Box([-1. -1. -8.], [1. 1. 8.], (3,), float32))

In [4]:
action_shape = env.action_space.shape[0]
action_shape

1

In [6]:
action_bound = [env.action_space.low,env.action_space.high]
action_bound

[array([-2.], dtype=float32), array([2.], dtype=float32)]

In [7]:
gamma = 0.9
tau = 0.001
replay_buffer = 10000
batch_size = 32

In [8]:
upper_bound = env.action_space.high[0]
lower_bound = env.action_space.low[0]

In [9]:
from DDPG import ddpg_keras as dd

In [10]:
std_dev = 0.2
ou_noise = dd.OUActionNoise(mean=np.zeros(1), std_deviation=float(std_dev) * np.ones(1))

In [11]:
ddpg = dd.DDPG(action_shape,state_shape,lower_bound,upper_bound,gamma,tau,buffer_max_len=5000,batch_size=200)

2022-02-01 23:59:24.744818: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (100)
2022-02-01 23:59:24.744875: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (MSI): /proc/driver/nvidia/version does not exist
2022-02-01 23:59:24.745208: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
num_episodes = 50
num_timesteps = 500

In [16]:
for i in range(num_episodes):
    state = env.reset()
    state = np.squeeze(state)
    state = state[np.newaxis,:]
    minibatch = []
    episode_avg_reward = 0
    for j in range(num_timesteps):
        action = ddpg.policy(state,ou_noise)
        next_state,reward,done,_ = env.step(action)
        episode_avg_reward += reward
        next_state = np.expand_dims(np.squeeze(next_state),0)
        #print(next_state)
        ddpg.replay_buffer(state,action,reward,next_state,done)
        state = next_state
    ddpg.train(batch_size=batch_size)
    print(f"Episode {i}: average reward: {episode_avg_reward/num_timesteps} ,total reward:{episode_avg_reward}")

Episode 0: average reward: -9.085268032887575 ,total reward:-4542.634016443787
Episode 1: average reward: -8.820533003610654 ,total reward:-4410.266501805327
Episode 2: average reward: -8.898488669046786 ,total reward:-4449.244334523393
Episode 3: average reward: -9.068600458909977 ,total reward:-4534.300229454989
Episode 4: average reward: -9.274167775856009 ,total reward:-4637.0838879280045
Episode 5: average reward: -9.171068598974857 ,total reward:-4585.534299487429
Episode 6: average reward: -8.898475574312807 ,total reward:-4449.237787156403
Episode 7: average reward: -8.80753511410177 ,total reward:-4403.7675570508845
Episode 8: average reward: -9.302035395799873 ,total reward:-4651.017697899936
Episode 9: average reward: -8.90111100507822 ,total reward:-4450.55550253911
Episode 10: average reward: -8.512629303810064 ,total reward:-4256.3146519050315
Episode 11: average reward: -8.673982460152741 ,total reward:-4336.991230076371
Episode 12: average reward: -8.558627170854272 ,to